# Exasol Personal Edition Demo

This notebook demonstrates connecting to an Exasol Personal Edition database instance and using some of its features. 

## 1. Open Secure Configuration Storage

First we need to open the Secure Configuration Storage (SCS) containing the connection information such as the database host, user, password, etc.

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui("../"))

## 2. Load UI functions

Additionally we will import some basic UI functions used in this notebook and activate the Magic Jupy SQL syntax.

In [ ]:
%run ../utils/access_store_ui.ipynb
%run ../utils/jupysql_init.ipynb

## 3. Create the Database Schema

We will create a separate database schema to contain our changes and experiments.

In [ ]:
%%sql --save udf_output
CREATE SCHEMA IF NOT EXISTS "{{ai_lab_config.db_schema}}"

AlteYou can also use the pyexasol module to create the database schema:

In [ ]:
from exasol.nb_connector.connections import open_pyexasol_connection
sql = f'CREATE SCHEMA IF NOT EXISTS "{ai_lab_config.db_schema}"'
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    conn.execute(query=sql)

## 4. Importing Data

This section demonstrates how to import data into the database. 

### 4.1 Create a Database Table

First we need to create a database table:


In [ ]:
%%sql --save udf_output
CREATE OR REPLACE TABLE "{{ai_lab_config.db_schema}}"."SALES" (
    sales_id DECIMAL(18,0),
    position_id INTEGER,
    article_id INTEGER,
    amount INTEGER,
    price DOUBLE,
    voucher_id INTEGER,
    canceled BOOL    
)

### 4.2 Import a CSV File

Now we can import a CSV file into the database table:

In [ ]:
with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    path = Path("sample.csv")
    import_params = {"column_separator": ";"}
    conn.import_from_file(path, (ai_lab_config.db_schema, "SALES"), import_params)

This does not work as import from local is only possible when using a JDBC driver.

```
%%sql 
import INTO "{{ai_lab_config.db_schema}}"."SALES"
FROM LOCAL csv file 'sample.csv' COLUMN separator = ';'
```

## 5. Using the Exasol Bucket File System

The [Exasol Bucket File System](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm) (BucketFS) is a powerful feature to exchange non-relational data with the database nodes in an Exasol cluster.

Such data can be arbitrary files including 
* Data to be processed by [User Defined Scripts](https://docs.exasol.com/db/latest/database_concepts/udf_scripts.htm) (UDFs)
* [Script-Language Containers](https://github.com/exasol/script-languages-release) (SLCs)
* Pretrained Large Language AI Models

### 5.1 Upload a File to the BucketFS

Let's upload a simple file into the BucketFS:

In [37]:
from exasol.nb_connector.connections import open_bucketfs_location
file = Path("text_file.txt")
bfs = open_bucketfs_location(ai_lab_config)
remote = bfs / file.name
remote.write(file.read_bytes())

### 5.2 List the Files in the BucketFS 

We can also list all the files currently available in the BucketFS:

In [ ]:
bfs = open_bucketfs_location(ai_lab_config)
for p in bfs.iterdir():
    print(f'- {p.name}')